In [1]:
import pandas as pd
import numpy as np
import vartools as vt
pd.set_option('display.float_format', '{:,.4f}'.format)


# Pregunta 1

In [9]:
def VaR_APL(df, positions, nc, long):
    bids = [bid for bid in df.columns if 'Bid' in bid]
    asks = [ask for ask in df.columns if 'Ask' in ask]

    last_mid = []

    for i in range(len(bids)):
        df[f'Mid{i}'] = (df[bids[i]] + df[asks[i]]) / 2
        df[f'Spread{i}'] = (df[asks[i]] - df[bids[i]]) / df[f'Mid{i}']
        df[f'Returns{i}'] = df[f'Mid{i}'].pct_change()
        last_mid.append(df[f'Mid{i}'].iloc[-1])

    weights = [weight for weight in (positions * last_mid) / ((last_mid * positions).sum())]
    df['Portfolio_Return'] = np.dot(df[[f'Returns{i}' for i in range(len(bids))]], weights)

    var = np.abs(np.percentile(df['Portfolio_Return'].dropna(), (100-nc)) if long else np.percentile(df['Portfolio_Return'].dropna(), nc))
    cvar = np.abs(df[df['Portfolio_Return'] < -var]['Portfolio_Return'].mean() if long else df[df['Portfolio_Return'] > var]['Portfolio_Return'].mean())

    cl_promedio = np.array([df[col].mean() for col in df.columns if 'Spread' in col])
    cl_estresado = np.array([np.percentile(df[col], 99) for col in df.columns if 'Spread' in col])

    var_apl_promedio, var_apl_estresado = var + cl_promedio.dot(weights), var + cl_estresado.dot(weights)
    cvar_apl_promedio, cvar_apl_estresado = cvar + cl_promedio.dot(weights), cvar + cl_estresado.dot(weights)

    cash_var, cash_cvar = var * (last_mid * positions).sum(), cvar * (last_mid * positions).sum()
    var_apl_promedio_cash, var_apl_estresado_cash = cash_var + cl_promedio.dot(positions*last_mid), cash_var + cl_estresado.dot(positions*last_mid)
    cvar_apl_promedio_cash, cvar_apl_estresado_cash = cl_promedio.dot(positions*last_mid) + cash_cvar, cash_cvar + cl_estresado.dot(positions*last_mid)

    categorias = ['VaR', 'CVaR', 'VaR APL Promedio', 'VaR APL Estresado', 'CVaR APL Promedio', 'CVaR APL Estresado']
    resultados = pd.DataFrame({
        'Métrica': categorias,
        '%': [var, cvar, var_apl_promedio, var_apl_estresado, cvar_apl_promedio, cvar_apl_estresado],
        '$': [cash_var, cash_cvar, var_apl_promedio_cash, var_apl_estresado_cash, cvar_apl_promedio_cash, cvar_apl_estresado_cash]
    })
    return resultados

# Pregunta 2

In [12]:
data = pd.read_excel('repasoexcel2.xlsx', sheet_name='Commodities')
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')
data = data.sort_index()

positions = np.array([121*100, 153*5000, 189*25000])
long = False
nc = 99
VaR_APL(data, positions, nc, long)

,Métrica,%,$
0,VaR,0.0244,"1,469,942.9064"
1,CVaR,0.0289,"1,738,382.5541"
2,VaR APL Promedio,0.0254,"1,529,102.2907"
3,VaR APL Estresado,0.0289,"1,740,885.0820"
4,CVaR APL Promedio,0.0299,"1,797,541.9383"
5,CVaR APL Estresado,0.0334,"2,009,324.7297"


In [27]:
pos = 121*100

ADV = np.mean(data['Volume'][-90:])
n_dias = np.ceil(pos / ADV)
n_dias

1.0

# Pregunta 2

In [23]:
df = pd.read_excel('repasoexcel2.xlsx', sheet_name='Forex').sort_values('Date').dropna(axis=1)
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

positions = np.array([1571000, 7203000])
long = True
nc = 99

VaR_APL(df, positions, nc, long)

,Métrica,%,$
0,VaR,0.0147,"46,302.2918"
1,CVaR,0.0171,"53,853.4672"
2,VaR APL Promedio,0.0150,"47,136.1542"
3,VaR APL Estresado,0.0153,"48,068.5437"
4,CVaR APL Promedio,0.0174,"54,687.3296"
5,CVaR APL Estresado,0.0177,"55,619.7191"


In [31]:
def VaR_APL(df, positions, nc, long):
    bids = [bid for bid in df.columns if 'Bid' in bid]
    asks = [ask for ask in df.columns if 'Ask' in ask]

    last_mid = []

    for i in range(len(bids)):
        df[f'Mid{i}'] = (df[bids[i]] + df[asks[i]]) / 2
        df[f'Spread{i}'] = (df[asks[i]] - df[bids[i]]) / df[f'Mid{i}']
        df[f'Returns{i}'] = df[f'Mid{i}'].pct_change()
        last_mid.append(df[f'Mid{i}'].iloc[-1])

    weights = [weight for weight in (positions * last_mid) / ((last_mid * positions).sum())]
    df['Portfolio_Return'] = np.dot(df[[f'Returns{i}' for i in range(len(bids))]], weights)

    var = np.abs(np.percentile(df['Portfolio_Return'].dropna(), (100-nc)) if long else np.percentile(df['Portfolio_Return'].dropna(), nc))
    cvar = np.abs(df[df['Portfolio_Return'] < -var]['Portfolio_Return'].mean() if long else df[df['Portfolio_Return'] > var]['Portfolio_Return'].mean())

    cl_promedio = np.array([df[col].mean() for col in df.columns if 'Spread' in col])
    cl_estresado = np.array([np.percentile(df[col], 99) for col in df.columns if 'Spread' in col])

    var_apl_promedio, var_apl_estresado = var + cl_promedio.dot(weights), var + cl_estresado.dot(weights)
    cvar_apl_promedio, cvar_apl_estresado = cvar + cl_promedio.dot(weights), cvar + cl_estresado.dot(weights)

    cash_var, cash_cvar = var * (last_mid * positions).sum(), cvar * (last_mid * positions).sum()
    var_apl_promedio_cash, var_apl_estresado_cash = cash_var + cl_promedio.dot(positions*last_mid), cash_var + cl_estresado.dot(positions*last_mid)
    cvar_apl_promedio_cash, cvar_apl_estresado_cash = cl_promedio.dot(positions*last_mid) + cash_cvar, cash_cvar + cl_estresado.dot(positions*last_mid)

    categorias = ['VaR', 'CVaR', 'VaR APL Promedio', 'VaR APL Estresado', 'CVaR APL Promedio', 'CVaR APL Estresado']
    resultados = pd.DataFrame({
        'Métrica': categorias,
        '%': [var, cvar, var_apl_promedio, var_apl_estresado, cvar_apl_promedio, cvar_apl_estresado],
        '$': [cash_var, cash_cvar, var_apl_promedio_cash, var_apl_estresado_cash, cvar_apl_promedio_cash, cvar_apl_estresado_cash]
    })
    return cl_promedio.dot(positions*last_mid)

In [32]:
VaR_APL(df, positions, nc, long)

833.8623314070503